In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import numpy as np
torch.manual_seed(1)

In [2]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

def prepare_sequence(seq, to_ix):
    oov_token = '<OOV>'
    idxs = [to_ix[w] if w in to_ix else to_ix[oov_token] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

# Load GloVe embeddings with an OOV token vector
def load_glove_embeddings(glove_file):
    word_to_ix = {}
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
    # Add OOV token to word_to_ix
    oov_token = '<OOV>'
    word_to_ix[oov_token] = len(word_to_ix)
    embedding_dim = len(next(iter(embeddings_index.values())))
    # Create embedding matrix including OOV token vector
    embedding_matrix = np.zeros((len(word_to_ix), embedding_dim))
    for word, i in word_to_ix.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            # Use random initialization for OOV token
            embedding_matrix[i] = np.random.normal(size=(embedding_dim,))
    return word_to_ix, torch.FloatTensor(embedding_matrix)


In [3]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, word_to_ix, tag_to_ix, embedding_dim, hidden_dim, embeddings=None):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.word_to_ix = word_to_ix
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        if embeddings is not None:
            self.word_embeds = nn.Embedding.from_pretrained(embeddings, freeze=False)
            self.embedding_dim = embeddings.size(1)
        else:
            self.word_embeds = nn.Embedding(len(word_to_ix), embedding_dim)

        self.lstm = nn.LSTM(self.embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        forward_var = init_alphas
        for feat in feats:
            alphas_t = []
            for next_tag in range(self.tagset_size):
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                trans_score = self.transitions[next_tag].view(1, -1)
                next_tag_var = forward_var + trans_score + emit_score
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []
            viterbivars_t = []
            for next_tag in range(self.tagset_size):
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):
        lstm_feats = self._get_lstm_features(sentence)
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [11]:
'''import json

filename = '/content/drive/MyDrive/NLP/pre-trained_embeddings/NER_train.json'

with open(filename, 'r') as file:
    data_train = json.load(file)

filename = '/content/drive/MyDrive/NLP/pre-trained_embeddings/NER_test.json'

with open(filename, 'r') as file:
    data_test = json.load(file)

filename = '/content/drive/MyDrive/NLP/pre-trained_embeddings/NER_val.json'

with open(filename, 'r') as file:
    data_val = json.load(file)'''

import json

filename = '/content/drive/MyDrive/NLP/Assignment-2_laptop/ATE_train.json'

with open(filename, 'r') as file:
    data_train = json.load(file)

filename = '/content/drive/MyDrive/NLP/Assignment-2_laptop/ATE_test.json'

with open(filename, 'r') as file:
    data_test = json.load(file)

filename = '/content/drive/MyDrive/NLP/Assignment-2_laptop/ATE_val.json'

with open(filename, 'r') as file:
    data_val = json.load(file)


In [12]:
data = data_val
tokenized_texts = []
labels = []

# Iterate over the items in the parsed JSON object
for key, value in data.items():
    # Extract text and labels for each item
    text = value['text']
    label_seq = value['labels']

    # Tokenize the text (if needed)
    tokenized_text = text.split()  # Using simple split for illustration

#     Store the tokenized text
    tokenized_texts.append(tokenized_text)
#     tokenized_texts.append(text)
    # Store the labels
    labels.append(label_seq)

val_data = []

for i in range(len(labels)):
    pair = [tokenized_texts[i], labels[i]]
    val_data.append(pair)

data = data_test
tokenized_texts = []
labels = []

# Iterate over the items in the parsed JSON object
for key, value in data.items():
    # Extract text and labels for each item
    text = value['text']
    label_seq = value['labels']

    # Tokenize the text (if needed)
    tokenized_text = text.split()  # Using simple split for illustration

#     Store the tokenized text
    tokenized_texts.append(tokenized_text)
#     tokenized_texts.append(text)
    # Store the labels
    labels.append(label_seq)

test_data = []

for i in range(len(labels)):
    pair = [tokenized_texts[i], labels[i]]
    test_data.append(pair)

data = data_train
tokenized_texts = []
labels = []

# Iterate over the items in the parsed JSON object
for key, value in data.items():
    # Extract text and labels for each item
    text = value['text']
    label_seq = value['labels']

    # Tokenize the text (if needed)
    tokenized_text = text.split()  # Using simple split for illustration

#     Store the tokenized text
    tokenized_texts.append(tokenized_text)
#     tokenized_texts.append(text)
    # Store the labels
    labels.append(label_seq)

train_data = []

for i in range(len(labels)):
    pair = [tokenized_texts[i], labels[i]]
    train_data.append(pair)

In [13]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {}
for tag_sen in labels:
    for tag in tag_sen:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

tag_to_ix[START_TAG] = len(tag_to_ix)
tag_to_ix[STOP_TAG] = len(tag_to_ix)
tag_to_ix

{'O': 0, 'B': 1, 'I': 2, '<START>': 3, '<STOP>': 4}

In [14]:
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

training_data = train_data
validation_data = val_data

# Initialize lists to store training and validation loss, and Macro-F1-score
train_losses = []
val_losses = []
train_f1_scores = []
val_f1_scores = []

glove_file = '/content/drive/MyDrive/NLP/pre-trained_embeddings/glove.6B.50d.txt'
word_to_ix, glove_embeddings = load_glove_embeddings(glove_file)
#model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, 50, 50, glove_embeddings)
#optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Training loop
'''for epoch in range(10):
    model.train()  # Set the model to training mode
    epoch_train_loss = 0.0
    y_true_train = []
    y_pred_train = []
    for sentence, tags in training_data:
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()

        # Record true and predicted tags for training F1-score
        with torch.no_grad():
            _, predicted_tags = model(sentence_in)
            y_true_train.extend(targets.numpy())
            y_pred_train.extend(predicted_tags)

    # Calculate training loss and F1-score
    train_loss = epoch_train_loss / len(training_data)
    train_losses.append(train_loss)
    train_f1 = f1_score(y_true_train, y_pred_train, average='macro')
    train_f1_scores.append(train_f1)

    # Validation
    model.eval()  # Set the model to evaluation mode
    epoch_val_loss = 0.0
    y_true_val = []
    y_pred_val = []
    for sentence, tags in validation_data:
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
        loss = model.neg_log_likelihood(sentence_in, targets)
        epoch_val_loss += loss.item()

        # Record true and predicted tags for validation F1-score
        with torch.no_grad():
            _, predicted_tags = model(sentence_in)
            y_true_val.extend(targets.numpy())
            y_pred_val.extend(predicted_tags)

    # Calculate validation loss and F1-score
    val_loss = epoch_val_loss / len(validation_data)
    val_losses.append(val_loss)
    val_f1 = f1_score(y_true_val, y_pred_val, average='macro')
    val_f1_scores.append(val_f1)
    print(f'Epoch [{epoch+1}/10], Train Loss: {epoch_train_loss:.4f}, Val Loss: {val_loss:.4f}, Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')'''


"for epoch in range(10):\n    model.train()  # Set the model to training mode\n    epoch_train_loss = 0.0\n    y_true_train = []\n    y_pred_train = []\n    for sentence, tags in training_data:\n        model.zero_grad()\n        sentence_in = prepare_sequence(sentence, word_to_ix)\n        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)\n        loss = model.neg_log_likelihood(sentence_in, targets)\n        loss.backward()\n        optimizer.step()\n        epoch_train_loss += loss.item()\n        \n        # Record true and predicted tags for training F1-score\n        with torch.no_grad():\n            _, predicted_tags = model(sentence_in)\n            y_true_train.extend(targets.numpy())\n            y_pred_train.extend(predicted_tags)\n    \n    # Calculate training loss and F1-score\n    train_loss = epoch_train_loss / len(training_data)\n    train_losses.append(train_loss)\n    train_f1 = f1_score(y_true_train, y_pred_train, average='macro')\n    train_f1

In [ ]:
# Plotting
'''
epochs = range(1, 11)  # 10 epochs

# Loss Plot
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

# F1 Plot
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_f1_scores, label='Training Macro-F1-score')
plt.plot(epochs, val_f1_scores, label='Validation Macro-F1-score')
plt.xlabel('Epochs')
plt.ylabel('Macro-F1-score')
plt.title('Training and Validation Macro-F1-score')
plt.legend()
plt.grid(True)
plt.show()'''

In [10]:
model1 = BiLSTM_CRF(len(word_to_ix), tag_to_ix, 50, 50, glove_embeddings)
device = torch.device('cpu')
model_path = '/content/drive/MyDrive/A2_19/Part-3/bi_lstm_crf_model_task1_GloVe.pth'
model1.load_state_dict(torch.load(model_path, map_location=device))

from sklearn.metrics import accuracy_score

test_losses = []
test_f1_scores = []

model1.eval()
y_true_test = []
y_pred_test = []
for sentence, tags in test_data:
    sentence_in = prepare_sequence(sentence, word_to_ix)
    targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
    #loss = model1.neg_log_likelihood(sentence_in, targets)
    #test_losses.append(loss.item())

    with torch.no_grad():
        _, predicted_tags = model1(sentence_in)
        y_true_test.extend(targets.numpy())
        y_pred_test.extend(predicted_tags)

#test_loss = sum(test_losses) / len(test_losses)
test_f1 = f1_score(y_true_test, y_pred_test, average='macro')
test_f1_scores.append(test_f1)
print(f'Test F1: {test_f1:.4f}')
print('Accuracy:', accuracy_score(y_true_test, y_pred_test))


Test F1: 0.4307
Accuracy: 0.903771600300526


In [15]:
model1 = BiLSTM_CRF(len(word_to_ix), tag_to_ix, 50, 50, glove_embeddings)
device = torch.device('cpu')
model_path = '/content/drive/MyDrive/A2_19/Part-3/bi_lstm_crf_model_task2_GloVe.pth'
model1.load_state_dict(torch.load(model_path, map_location=device))

from sklearn.metrics import accuracy_score

test_losses = []
test_f1_scores = []

model1.eval()
y_true_test = []
y_pred_test = []
for sentence, tags in test_data:
    sentence_in = prepare_sequence(sentence, word_to_ix)
    targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
    #loss = model1.neg_log_likelihood(sentence_in, targets)
    #test_losses.append(loss.item())

    with torch.no_grad():
        _, predicted_tags = model1(sentence_in)
        y_true_test.extend(targets.numpy())
        y_pred_test.extend(predicted_tags)

#test_loss = sum(test_losses) / len(test_losses)
test_f1 = f1_score(y_true_test, y_pred_test, average='macro')
test_f1_scores.append(test_f1)
print(f'Test F1: {test_f1:.4f}')
print('Accuracy:', accuracy_score(y_true_test, y_pred_test))


Test F1: 0.7310
Accuracy: 0.9168439373428737
